In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [2]:
conf = SparkConf()
spark_context = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
spark = SparkSession.builder.master("spark://spark-master:7077").config("spark.jars.packages", 
                                                                        "org.apache.hadoop:hadoop-aws-2.7.3").appName("spark-example").getOrCreate()

## Load data

### Load CSV file into DataFrame


In [4]:
csv_file = "../data/oscars.csv"
df_csv = spark.read.csv(csv_file, header=True, inferSchema=True)

df_csv.show()

+--------+--------+------------+-------------------+--------------------+---+-------------+----------+--------+-------+-------+
|oscar_no|oscar_yr|       award|               name|               movie|age|     birth_pl|birth_date|birth_mo|birth_d|birth_y|
+--------+--------+------------+-------------------+--------------------+---+-------------+----------+--------+-------+-------+
|       1|    1929|Best actress|       Janet Gaynor|          7th Heaven| 22| Pennsylvania|1906-10-06|      10|      6|   1906|
|       2|    1930|Best actress|      Mary Pickford|            Coquette| 37|       Canada|1892-04-08|       4|      8|   1892|
|       3|    1931|Best actress|      Norma Shearer|        The Divorcee| 28|       Canada|1902-08-10|       8|     10|   1902|
|       4|    1932|Best actress|     Marie Dressler|        Min and Bill| 63|       Canada|1868-11-09|      11|      9|   1868|
|       5|    1933|Best actress|        Helen Hayes|The Sin of Madelo...| 32|Washington DC|1900-10-10|  

In [5]:
df_csv.explain(True)

== Parsed Logical Plan ==
Relation [oscar_no#17,oscar_yr#18,award#19,name#20,movie#21,age#22,birth_pl#23,birth_date#24,birth_mo#25,birth_d#26,birth_y#27] csv

== Analyzed Logical Plan ==
oscar_no: int, oscar_yr: int, award: string, name: string, movie: string, age: int, birth_pl: string, birth_date: date, birth_mo: int, birth_d: int, birth_y: int
Relation [oscar_no#17,oscar_yr#18,award#19,name#20,movie#21,age#22,birth_pl#23,birth_date#24,birth_mo#25,birth_d#26,birth_y#27] csv

== Optimized Logical Plan ==
Relation [oscar_no#17,oscar_yr#18,award#19,name#20,movie#21,age#22,birth_pl#23,birth_date#24,birth_mo#25,birth_d#26,birth_y#27] csv

== Physical Plan ==
FileScan csv [oscar_no#17,oscar_yr#18,award#19,name#20,movie#21,age#22,birth_pl#23,birth_date#24,birth_mo#25,birth_d#26,birth_y#27] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/D:/RobotDream/homework/lec14/spark01/data/oscars.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<

In [6]:
df_csv.printSchema()

root
 |-- oscar_no: integer (nullable = true)
 |-- oscar_yr: integer (nullable = true)
 |-- award: string (nullable = true)
 |-- name: string (nullable = true)
 |-- movie: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- birth_pl: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- birth_mo: integer (nullable = true)
 |-- birth_d: integer (nullable = true)
 |-- birth_y: integer (nullable = true)



In [7]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("year", StringType(), True),
    StructField("age", IntegerType(), True) #Wrong type
])

df_csv_with_schema = spark.read.schema(schema).csv(csv_file, header=True)

df_csv_with_schema.show()

+---+----+----+
| id|year| age|
+---+----+----+
|  1|1929|NULL|
|  2|1930|NULL|
|  3|1931|NULL|
|  4|1932|NULL|
|  5|1933|NULL|
|  6|1934|NULL|
|  7|1935|NULL|
|  8|1936|NULL|
|  9|1937|NULL|
| 10|1938|NULL|
| 11|1939|NULL|
| 12|1940|NULL|
| 13|1941|NULL|
| 14|1942|NULL|
| 15|1943|NULL|
| 16|1944|NULL|
| 17|1945|NULL|
| 18|1946|NULL|
| 19|1947|NULL|
| 20|1948|NULL|
+---+----+----+
only showing top 20 rows



In [8]:
df_csv_with_schema.printSchema()

root
 |-- id: integer (nullable = true)
 |-- year: string (nullable = true)
 |-- age: integer (nullable = true)



In [9]:
# Load CSV file with specific options
df_csv_options = spark.read.options(header='True', 
                                    inferSchema='True', 
                                    delimiter=',',
                                    quote='"',
                                    dateFormat='yyyy-MM-dd',
                                    escape='\\').csv(csv_file)

df_csv_options.show()

+--------+--------+------------+-------------------+--------------------+---+-------------+----------+--------+-------+-------+
|oscar_no|oscar_yr|       award|               name|               movie|age|     birth_pl|birth_date|birth_mo|birth_d|birth_y|
+--------+--------+------------+-------------------+--------------------+---+-------------+----------+--------+-------+-------+
|       1|    1929|Best actress|       Janet Gaynor|          7th Heaven| 22| Pennsylvania|1906-10-06|      10|      6|   1906|
|       2|    1930|Best actress|      Mary Pickford|            Coquette| 37|       Canada|1892-04-08|       4|      8|   1892|
|       3|    1931|Best actress|      Norma Shearer|        The Divorcee| 28|       Canada|1902-08-10|       8|     10|   1902|
|       4|    1932|Best actress|     Marie Dressler|        Min and Bill| 63|       Canada|1868-11-09|      11|      9|   1868|
|       5|    1933|Best actress|        Helen Hayes|The Sin of Madelo...| 32|Washington DC|1900-10-10|  

### Load JSON file into DataFrame

In [10]:
json_file = "../data/sales.json"
df_json = spark.read.json(json_file, multiLine=True)

df_json.show()

+-----------------+-----+--------------+-------------+
|           client|price|       product|purchase_date|
+-----------------+-----+--------------+-------------+
|     Norma Fisher|  121|Vacuum cleaner|   2022-08-09|
|     Norma Fisher|  348|Microwave oven|   2022-08-13|
|     Norma Fisher| 1126|         Phone|   2022-08-12|
|   Jorge Sullivan|  171|Microwave oven|   2022-08-10|
|  Elizabeth Woods| 1766|            TV|   2022-08-26|
|     Susan Wagner|  461|Microwave oven|   2022-08-26|
|     Susan Wagner|  561|Microwave oven|   2022-08-05|
| Peter Montgomery| 1994|            TV|   2022-08-03|
| Peter Montgomery| 2804|coffee machine|   2022-08-16|
|Stephanie Collins|  403|Vacuum cleaner|   2022-08-20|
|Stephanie Collins| 1775|coffee machine|   2022-08-18|
| Stephanie Sutton|  613|Vacuum cleaner|   2022-08-09|
| Stephanie Sutton| 2148|            TV|   2022-08-30|
| Stephanie Sutton|  568|Microwave oven|   2022-08-01|
|       Susan Levy|  109|coffee machine|   2022-08-20|
|       Su

In [ ]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("details", StructType([
        StructField("age", IntegerType(), True),
        StructField("address", StructType([
            StructField("city", StringType(), True),
            StructField("state", StringType(), True)
        ]), True)
    ]), True)
])

nested_json_file = "../data/few_files/*.json"
df_nested_json = spark.read.schema(schema).json(nested_json_file, multiLine=True)

df_nested_json.show(truncate=False)

In [ ]:
df_nested_json.printSchema()

### Load Parquet file into DataFrame

In [ ]:
parquet_file = "../data/titanic.parquet"
df_parquet = spark.read.parquet(parquet_file)

# Show DataFrame
df_parquet.show()

### Load Text file into DataFrame

In [ ]:
text_file = "../data/android.txt"
df_text = spark.read.text(text_file)

# Show DataFrame
df_text.show()

In [ ]:
sc = spark.sparkContext

# Load text file into an RDD
rdd = sc.textFile(text_file)

In [ ]:
rdd.take(10)

### Loading Data from Apache Kafka

In [ ]:
# Initialize Spark session with Kafka package
spark = SparkSession.builder \
    .appName("Load Kafka") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1") \
    .getOrCreate()

# Load data from Kafka
kafka_df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "your_topic") \
    .load()

kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").show()

### Loading Data from a JDBC Source with Partitioning

In [ ]:
# Load data from a SQL database with partitioning
jdbc_url = "jdbc:postgresql://your-db-host:5432/your-database"
properties = {
    "user": "your-username",
    "password": "your-password",
    "driver": "org.postgresql.Driver"
}

df_sql_partitioned = spark.read.jdbc(
    url=jdbc_url,
    table="your_table_name",
    properties=properties,
    column="id",
    lowerBound=1,
    upperBound=100000,
    numPartitions=10
)

df_sql_partitioned.show()


### RDD

In [ ]:
rdd.take(10)

In [ ]:
line_count = rdd.count()
print(f"Number of lines: {line_count}")



In [ ]:
first_line = rdd.first()
print(f"First line: {first_line}")

In [ ]:
filtered_rdd = rdd.filter(lambda line: 'WindowManager:' in line)
filtered_rdd.take(10)

In [ ]:
line_lengths = rdd.map(lambda line: len(line))
line_lengths.take(10)

In [ ]:
# Reduce to get the total number of characters in the text file
total_characters = line_lengths.reduce(lambda a, b: a + b)
print(f"Total number of characters: {total_characters}")

In [ ]:
rdd.flatMap(lambda line: line.split(" ")).take(10)

In [ ]:
words = rdd.flatMap(lambda line: line.split(" "))
word_pairs = words.map(lambda word: (word, 1))
word_counts = word_pairs.reduceByKey(lambda a, b: a + b)

word_counts.take(10)

In [ ]:
schema = StructType([
    StructField("word", StringType(), True),
    StructField("count", IntegerType(), True)
])

word_counts_df = spark.createDataFrame(word_counts, schema).show()


In [ ]:
import re

In [ ]:
# Define the regular expression pattern for real words
pattern = re.compile(r'\b[A-Za-z]+\b')

def extract_words(line):
    return pattern.findall(line)

words_rdd = rdd.flatMap(extract_words)

words_rdd.take(10)

In [ ]:
word_pairs = words_rdd.map(lambda word: (word, 1))
word_counts = word_pairs.reduceByKey(lambda a, b: a + b)

word_counts.sortByKey().take(20)

In [ ]:
word_counts.sortBy(lambda x: x[1], ascending=False).take(20)

In [ ]:
word_counts.distinct().take(20)

In [ ]:
word_counts.count()

In [ ]:
word_counts.collect()

word_counts.getNumPartitions()

In [ ]:
rdd1 = sc.parallelize([("a", 1), ("b", 2)])
rdd2 = sc.parallelize([("a", 3), ("b", 4), ("c", 5)])
result = rdd1.join(rdd2)
result.take(10)

In [ ]:
rdd1 = sc.parallelize([1, 2, 3])
rdd2 = sc.parallelize([3, 4, 5])
result = rdd1.union(rdd2)

result.take(10)

### DataFrame

In [ ]:
df_parquet.select("name").show()
df_parquet.select("name", "age").show()

In [ ]:
df_parquet.filter(df_parquet["age"] > 25).show()

In [ ]:
df_parquet.filter("age > 25").show()

In [ ]:
df_parquet.filter(F.col("age") > 25).show()

In [ ]:
df_parquet.filter((F.col("age") > 25) & (F.col("age") < 30)).show()

In [ ]:
df_grouped = df_parquet.groupBy("age").count()
df_grouped.show()

In [ ]:
df_parquet.groupBy("age")

In [ ]:
df_grouped = df_parquet.groupBy("age").avg("Fare")
df_grouped.show()

In [ ]:
df_parquet.agg(F.avg("age")).show()

In [ ]:
df_parquet.agg(F.min("age")).show()

In [ ]:
df_parquet.agg(F.max("age")).show()

In [ ]:
df_parquet.withColumn("age_in_10_years", F.col("age") + 10).show()

In [ ]:
data = [("John", 30), ("Jane", 25), ("Doe", 22)]
df = spark.createDataFrame(data, ["name", "age"])

data2 = [("John", "USA"), ("Jane", "UK"), ("Doe", "Canada")]
df2 = spark.createDataFrame(data2, ["name", "country"])

df_joined = df.join(df2, on="name", how="inner")
df_joined.show()

In [ ]:
df_parquet.orderBy("age").show()
df_parquet.orderBy(F.col("age").desc()).show()

In [ ]:
df_parquet.drop("age").show()

In [ ]:
df_parquet.show()

In [ ]:
df_parquet.distinct().show()

In [ ]:
df_parquet.dropDuplicates().show()

In [ ]:
df_parquet.dropDuplicates(["age"]).show()

In [ ]:
df_parquet.fillna({"age": 0}).show()

In [ ]:
df_parquet.replace({3: 30, 1: 10}, subset='Pclass').show()

In [ ]:
df_parquet.sample(fraction=0.5).show()

In [ ]:
data1 = [("John", 30), ("Jane", 25)]
data2 = [("John", 30), ("Doe", 22)]
df1 = spark.createDataFrame(data1, ["name", "age"])
df2 = spark.createDataFrame(data2, ["name", "age"])

df1.intersect(df2).show()

In [ ]:
data1 = [("John", 30), ("Jane", 25)]
data2 = [("Doe", 22), ("Smith", 35)]
df1 = spark.createDataFrame(data1, ["name", "age"])
df2 = spark.createDataFrame(data2, ["name", "age"])

df1.union(df2).show()

In [ ]:
data1 = [("John", 30), ("Jane", 25)]
data2 = [("John", 30), ("Doe", 22)]
df1 = spark.createDataFrame(data1, ["name", "age"])
df2 = spark.createDataFrame(data2, ["name", "age"])

df1.exceptAll(df2).show()

In [ ]:
df_parquet.describe().show()

In [ ]:
df_parquet.select(df_parquet["name"].alias("full_name"), "age").show()

In [ ]:
pandas_df = df_parquet.toPandas()
pandas_df

In [ ]:
df_parquet.withColumnRenamed("name", "full_name").show()

In [ ]:
df_parquet.sort(df_parquet["age"].desc()).show()

In [ ]:
df_parquet.rdd.take(
    10
)

In [ ]:
df_parquet.limit(2).show()

In [ ]:
# df_parquet.head(2)

In [ ]:
for part in df_parquet.randomSplit([0.3, 0.4, 0.3]):
    part.show()


In [ ]:
df_parquet.write.mode("overwrite").parquet("./data/output/new_data.parquet")

In [ ]:
df_parquet.filter(F.col("age").isNotNull()).show()

In [ ]:
df_parquet.filter(F.col("age").isNull()).show()

In [ ]:
df_parquet.dropna().show()

In [ ]:
df_parquet.select(F.countDistinct("name")).show()


In [ ]:
df_parquet.corr("age", "Fare")

In [ ]:
df_parquet.cov("age", "Fare")

In [ ]:
quantiles = df_parquet.approxQuantile("age", [0.25, 0.5, 0.75], 0.01)
print(f"Approximate quantiles: {quantiles}")

In [ ]:
df_parquet.sampleBy("name", fractions={"a": 0.5, "b": 1.0, "c": 0.2}).show()

In [ ]:
df_parquet.rollup("age").sum("Fare").show()

In [ ]:
def add_years(df):
    return df.withColumn("age_plus_5", F.col("age") + 5)
    
df_parquet.transform(add_years).show()

In [ ]:
data1 = [("John", 30), ("Jane", 25)]
data2 = [("USA",), ("UK",)]
df1 = spark.createDataFrame(data1, ["name", "age"])
df2 = spark.createDataFrame(data2, ["country"])

# Perform a cross join
df_cross_join = df1.crossJoin(df2)
df_cross_join.show()

In [ ]:
df_parquet.createOrReplaceTempView("people")

# Run SQL queries on the temporary view
spark.sql("SELECT * FROM people WHERE age > 25").show()

In [ ]:
df_parquet.selectExpr("name as full_name", "age + 5 as age_in_5_years").show()

In [ ]:
df_parquet.inputFiles()

In [ ]:
df_parquet.isEmpty()

In [ ]:
df_parquet.isStreaming

In [ ]:
df_parquet.summary().show()

In [ ]:
window_spec = Window.partitionBy("sex").orderBy("Survived")

In [ ]:
# Apply row_number
df_parquet.withColumn("row_number", F.row_number().over(window_spec)).where("row_number = 1").show()

# Apply rank
df_parquet.withColumn("rank", F.rank().over(window_spec)).where("rank = 1").show()

# Apply dense_rank
df_parquet.withColumn("dense_rank", F.dense_rank().over(window_spec)).where(F.col("dense_rank").isin([1, 2])).show()


In [ ]:
# Filter names that contain the substring 'a'
df_parquet.filter(F.col("name").like("%a%")).show()

# Filter names that start with 'A'
df_parquet.filter(F.col("name").like("A%")).show()

# Filter names that end with 'e'
df_parquet.filter(F.col("name").like("%e")).show()

In [ ]:
df_parquet.filter(F.col("name").endswith("A")).show()

df_parquet.filter(F.col("name").startswith("D")).show()

In [ ]:
spark.stop()